In [25]:
%reload_ext autoreload
%autoreload 2

import os
import argparse
import sys
from time import time
import cv2

from joblib import Parallel, delayed

sys.path.append(os.path.join(os.environ['GORDON_REPO_DIR'], 'utilities'))
from utilities2015 import *

from matplotlib.path import Path
%matplotlib inline

import scipy.ndimage as nd
import scipy
from sklearn.cluster import KMeans
import random
from multiprocessing import Pool

In [19]:
# 1. Random sample from training images
# 2. Perform k-means on their features
trainingDir = '/oasis/projects/nsf/csd395/yuncong/CSHL_data_patches/MD589_byLandmark/';
savingFileName = '/oasis/projects/nsf/csd395/ruogu/svm3';
M = 200;

In [16]:
# Input: rgb color image
def getFeaturesFromImage(image, patchSize=16, stride=56):
    # Change image to gray scale
    img = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY);
    # Generate key points
    height = img.shape[0];
    width = img.shape[1];
    xv, yv = np.meshgrid(np.arange(patchSize / 2, width - patchSize / 2, stride),
                         np.arange(patchSize / 2, height - patchSize / 2, stride),
                         indexing='ij');
    sample_points = np.c_[xv.flat, yv.flat];
    cv_keypoints = [cv2.KeyPoint(x,y, patchSize) for x, y in sample_points];
    sift = cv2.SIFT();
    _, descriptors = sift.compute(img, cv_keypoints);
    return descriptors;

In [22]:
def getFeaturesFromDir(directory):
    fileNames = os.listdir(directory);
    if len(fileNames) > 100:
        fileNames = random.sample(fileNames, 100);
    feature_list = [];
    for fileName in fileNames:
        image = imread(directory + fileName);
        feature_list += getFeaturesFromImage(image).tolist();
    return feature_list;

In [29]:
subDirNames = os.listdir(trainingDir);
dir_list = [trainingDir + x + '/' for x in subDirNames];
t = time();
pool = Pool(processes=8);
result = pool.map(getFeaturesFromDir, dir_list);
print "Gathering features takes {} sec.".format(time() - t);

11.7760801315


In [31]:
feature_bank = [];
for x in result:
    feature_bank += x;
feature_bank = np.asarray(feature_bank);
print '%s' %(feature_bank.shape,);

(28800, 128)


In [36]:
t = time();
kmeans = KMeans(init='random', n_clusters=M, n_init=10);
kmeans.fit(feature_bank);
print "K-means takes {} sec.".format(time() - t);

K-means takes 850.37383604 sec.


In [37]:
np.save('/oasis/projects/nsf/csd395/ruogu/svm3/vocabulary.npy', kmeans.cluster_centers_);